In [455]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp

In [456]:
df = pd.read_csv("./data/t3p1.csv")
# Columns of df are: 'power_aimed', 'voltage_pv', 'current_pv', 'voltage_batt', 'current_batt', 'voltage_inv_dc', 'current_inv_dc', 'power_inv_ac', 'power_pv', 'power_batt', 'power_inv_dc', 'controller_type'
# Recalculate powers
df['power_batt'] = df['voltage_batt'] * df['current_batt']
df['power_inv_dc'] = df['voltage_inv_dc'] * df['current_inv_dc']
df['power_pv'] = df['voltage_pv'] * df['current_pv']


# Calculate efficiencies
# efficiency_controller = (power_batt + power_inv_dc) / power_pv
# efficiency_inverter = power_inv_ac / power_inv_dc
df['efficiency_controller'] = (df['power_batt'] + df['power_inv_dc']) / df['power_pv']
df['efficiency_inverter'] = df['power_inv_ac'] / df['power_inv_dc']

# Set up the dataframes
dfpwm = df.copy(deep=True)
dfmppt = df.copy(deep=True)
dfpwm = dfpwm[dfpwm['controller_type'] == 'PWM']
dfmppt = dfmppt[dfmppt['controller_type'] == 'MPPT']

In [457]:
dfpwm.head(10)

,power_aimed,voltage_pv,current_pv,voltage_batt,current_batt,voltage_inv_dc,current_inv_dc,power_inv_ac,power_pv,power_batt,power_inv_dc,controller_type,efficiency_controller,efficiency_inverter
0,2.0,13.13,4.76,13.87,3.68,14.09,0.64,2.2,62.4988,51.0416,9.0176,PWM,0.960966,0.243967
1,5.0,13.33,4.76,13.85,3.48,13.97,1.01,6.6,63.4508,48.1980,14.1097,PWM,0.981984,0.467763
2,10.0,13.37,4.76,13.79,3.20,13.87,1.32,10.5,63.6412,44.1280,18.3084,PWM,0.981069,0.573507
3,15.0,13.72,4.76,13.69,2.85,13.75,1.66,15.1,65.3072,39.0165,22.8250,PWM,0.946932,0.661555
4,20.0,13.18,4.76,13.60,2.49,13.60,2.02,19.9,62.7368,33.8640,27.4720,PWM,0.977672,0.724374
5,25.0,13.18,4.76,13.51,2.20,13.48,2.53,25.4,62.7368,29.7220,34.1044,PWM,1.017368,0.744772
6,30.0,13.74,4.76,13.43,1.67,13.31,3.06,30.4,65.4024,22.4281,40.7286,PWM,0.965663,0.746404
7,35.0,13.62,4.76,13.34,1.20,13.16,3.52,34.6,64.8312,16.0080,46.3232,PWM,0.961438,0.746926
8,40.0,13.52,4.76,13.25,0.68,13.00,4.04,40.6,64.3552,9.0100,52.5200,PWM,0.956100,0.773039
9,45.0,13.40,4.76,13.16,0.22,12.85,4.51,45.7,63.7840,2.8952,57.9535,PWM,0.953981,0.788563


In [458]:
# Get index of the first point where power_batt < 0
pwm_switch_index = dfpwm[dfpwm['power_batt'] < 0].index[0]
marker_size = 14

dfmppt = dfmppt.reset_index(drop=True)
mppt_switch_index = dfmppt[dfmppt['power_batt'] < 0].index[0]

marker_opacity = 0.85

# For pwm controller, plot with subplots
# x-axis: power_inv_dc
# y-axis left (primary): efficiency_controller
# y-axis right (secondary): efficiency_inverter
# Create a vertical line through first point where power_batt < 0

fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
# fig = go.Figure()
fig.add_trace(go.Scatter(x=dfpwm['power_inv_dc'], y=dfpwm['efficiency_controller'], name='PR1010 Controller', mode='markers',marker=dict(size=marker_size, symbol='circle', line=dict(width=2, color='black'), opacity = marker_opacity)), secondary_y=False)
fig.add_trace(go.Scatter(x=dfpwm['power_inv_dc'], y=dfpwm['efficiency_inverter'], name='PR1010 Inverter', mode='lines+markers', marker=dict(size=marker_size*0.8, symbol='circle-dot', line=dict(width=2, color='DarkSlateGray'), opacity = marker_opacity*0.8)), secondary_y=True)
fig.add_vline(x=dfpwm['power_inv_dc'].iloc[pwm_switch_index], line_width=3, line_dash="dash", line_color="green", opacity=0.66)

fig.add_trace(go.Scatter(x=dfmppt['power_inv_dc'], y=dfmppt['efficiency_controller'], name='MPPT2010 Controller', mode='markers', marker=dict(size=marker_size, symbol='diamond', line=dict(width=2, color='black'), opacity = marker_opacity)), secondary_y=False)
fig.add_trace(go.Scatter(x=dfmppt['power_inv_dc'], y=dfmppt['efficiency_inverter'], name='MPPT2010 Inverter', mode='lines+markers', marker=dict(size=marker_size*0.8, symbol='diamond-dot', line=dict(width=2, color='DarkSlateGray'), opacity = marker_opacity*0.8)), secondary_y=True)
fig.add_vline(x=dfmppt['power_inv_dc'].iloc[mppt_switch_index], line_width=3, line_dash="longdashdot", line_color="orange", opacity=0.66)


fig.update_layout(title='PWM Controller and Inverter Efficiencies', xaxis_title='Power Inverter DC [W]', yaxis_title='Efficiency [%]')
fig.update_yaxes(title_text="Controller Efficiency", secondary_y=False)
fig.update_yaxes(title_text="Inverter Efficiency", secondary_y=True)
# Update font to 20
fig.update_layout(font=dict(size=20))
# Update font of title to 30
fig.update_layout(title_font=dict(size=30))

# Update size to 800x800
fig.update_layout(width=800,height=800)
# Move legend to top right
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.0,
    xanchor="right",
    x=0.95
))
# Update title
fig.update_layout(title=dict(y=0.99))
# Add outline to legend
# fig.update_layout(legend=dict(
#     bordercolor="Black",
#     borderwidth=1
# ))
# Show the plot

# Annotate the vertical lines with texts MPPT or PWM and the power at which the switch occurs
fig.add_annotation(x=dfpwm['power_inv_dc'].iloc[pwm_switch_index], y=0.84, text="PWM \nBattery Discharge", showarrow=True, arrowhead=1, ax=-145, ay=-35, font=dict(size=21, color='green'), bgcolor='white')
fig.add_annotation(x=dfmppt['power_inv_dc'].iloc[mppt_switch_index], y=0.85, text="MPPT \nBattery Discharge", showarrow=True, arrowhead=1, ax=-205, ay=-65, font=dict(size=21, color='orange'), bgcolor='white')

fig.show()


In [459]:
# Get index of the first point where power_batt < 0
# Reindex dfmppt
dfmppt = dfmppt.reset_index(drop=True)
mppt_switch_index = dfmppt[dfmppt['power_batt'] < 0].index[0]

# For mppt controller:
# x-axis: power_inv_dc
# y-axis left (primary): efficiency
# Plot controller as circle and inverter as cross with outlines
# Create a vertical line through first point where power_batt < 0

# fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfmppt['power_inv_dc'], y=dfmppt['efficiency_controller'], name='MPPT Controller', mode='markers',marker=dict(size=marker_size, symbol='circle', line=dict(width=2, color='black'))))
fig.add_trace(go.Scatter(x=dfmppt['power_inv_dc'], y=dfmppt['efficiency_inverter'], name='MPPT Inverter', mode='markers', marker=dict(size=marker_size, symbol='cross', line=dict(width=2, color='black'))))
fig.add_vline(x=dfmppt['power_inv_dc'].iloc[mppt_switch_index], line_width=3, line_dash="dash", line_color="green", opacity=0.66)
fig.update_layout(title='MPPT Controller and Inverter Efficiencies', xaxis_title='Power Inverter DC [W]', yaxis_title='Efficiency [%]')
fig.update_yaxes(title_text="Efficiency")
# Update font to 20
fig.update_layout(font=dict(size=20))
# Update size to 800x800
fig.update_layout(width=800,height=800)
# Move legend to top right
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
# Add outline to legend
# fig.update_layout(legend=dict(
#     bordercolor="Black",
#     borderwidth=2
# ))
# Show the plot
fig.show()

